In [1]:
import keras
import tensorflow as tf 
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.utils import image_dataset_from_directory
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  0
[]


In [4]:
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())

TensorFlow version: 2.16.1
Built with CUDA: False


In [5]:
base_model = keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3)) #input has it be 244, 244, 3 

In [6]:
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
base_model.trainable = False

In [8]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)
x = Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x)
x = Dropout(0.7)(x)
x = Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x)
outputs = Dense(32, activation="softmax")(x)

In [9]:
model = Model(inputs, outputs)

In [10]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [11]:
train_ds = image_dataset_from_directory("NHL_LOGOS", labels="inferred", label_mode="categorical", image_size=(224, 224), batch_size=8, shuffle=True, seed=42, validation_split=0.2, subset="training")

Found 644 files belonging to 32 classes.
Using 516 files for training.


In [12]:
val_ds = image_dataset_from_directory("NHL_LOGOS", labels="inferred", label_mode="categorical", image_size=(224, 224), batch_size=8, shuffle=True, seed=42, validation_split=0.2, subset="validation")

Found 644 files belonging to 32 classes.
Using 128 files for validation.


In [13]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.3),
    keras.layers.RandomZoom(0.3),
    keras.layers.RandomContrast(0.2),
    keras.layers.RandomBrightness(0.2)
        ])

In [14]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

In [15]:
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

In [16]:
Autotune = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=Autotune)
val_ds = val_ds.cache().prefetch(buffer_size=Autotune)

In [18]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3, callbacks=[early_stop, reduce_lr])

Epoch 1/3
65/65 ━━━━━━━━━━━━━━━━━━━━ 31s 477ms/step - accuracy: 0.6899 - loss: 7.6140 - val_accuracy: 0.9453 - val_loss: 6.8843 - learning_rate: 1.0000e-04
Epoch 2/3
65/65 ━━━━━━━━━━━━━━━━━━━━ 31s 475ms/step - accuracy: 0.8043 - loss: 6.6559 - val_accuracy: 0.9766 - val_loss: 6.0631 - learning_rate: 1.0000e-04
Epoch 3/3
65/65 ━━━━━━━━━━━━━━━━━━━━ 31s 474ms/step - accuracy: 0.8857 - loss: 5.9279 - val_accuracy: 0.9766 - val_loss: 5.4730 - learning_rate: 1.0000e-04


In [19]:
model.save("nhl_logo_classifier_model.keras")